In [ ]:
import sys
import os
_path = os.path.abspath('../scripts/')
if _path not in sys.path:
    sys.path.append(_path)

import astropy.coordinates as coord
coord.galactocentric_frame_defaults.set('v4.0')
import astropy.table as at
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from scipy.special import logsumexp

from pyia import GaiaData
import theano
# theano.config.gcc.cxxflags = "-fbracket-depth=16000" # default is 256 - only need this on Mac??
import pymc3 as pm
import theano.tensor as tt
import exoplanet as xo

from model import FieldHelper

Expected mean disk velocity in ICRS:

In [ ]:
xtmp = coord.CartesianRepresentation(100, 0, 0, unit=u.km)
vlsr = coord.CartesianDifferential(-(np.array([12.9, 245.6, 7.78]) - np.array([0, 234, 0])) * u.km/u.s)
vthin = coord.SkyCoord(xtmp.with_differentials(vlsr), frame='galactic').icrs.velocity.d_xyz.value

xtmp = coord.CartesianRepresentation(100, 0, 0, unit=u.km)
vlsr = coord.CartesianDifferential(-(np.array([12.9, 245.6, 7.78]) - np.array([0, 200, 0])) * u.km/u.s)
vthik = coord.SkyCoord(xtmp.with_differentials(vlsr), frame='galactic').icrs.velocity.d_xyz.value

xtmp = coord.CartesianRepresentation(100, 0, 0, unit=u.km)
vsun = coord.CartesianDifferential(-np.array([12.9, 245.6, 7.78]) * u.km/u.s)
vhalo = coord.SkyCoord(xtmp.with_differentials(vsun), frame='galactic').icrs.velocity.d_xyz.value

vthin, vthik, vhalo

In [ ]:
vfield0 = np.stack([vthin, vthik, vhalo])
sigvfield0 = np.stack([20., 50., 125.])

In [ ]:
g = GaiaData('../data/150pc_MG12-result.fits.gz')

In [ ]:
subg = g[np.isfinite(g.radial_velocity)]
# subg = g
subg = subg[np.random.choice(len(subg), size=1024, replace=False)]
np.isfinite(subg.radial_velocity).sum()

In [ ]:
for gg in [g, subg]:
    c = gg.get_skycoord()

    fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)
    for i in range(3):
        ax = axes[i]

        _x = c.velocity.d_xyz.value[i]
        print(1.5 * np.nanmedian(np.abs(_x - np.nanmedian(_x))),
              np.nanmedian(_x))
        ax.hist(c.velocity.d_xyz.value[i],
                bins=np.linspace(-150, 150, 64), alpha=0.4, label=str(i));

        ax.axvline(vthin[i], color='tab:red', linestyle='--')
        ax.axvline(vthik[i], color='tab:green', linestyle='--')

    ax.legend(loc='best')
#     ax.set_yscale('log')

    fig.tight_layout()

In [ ]:
from model import BaseHelper, UniformSpaceDensity, pc_mas_yr_per_km_s
class FieldHelper(BaseHelper):

    def get_model(self, vfield0, sigma_vfield0):
        # Number of prior mixture components:
        with pm.Model() as model:

            # True distance:
            rlim = 250.
            BoundedR = pm.Bound(UniformSpaceDensity, lower=0, upper=rlim)
            r = BoundedR("r", rlim, shape=(self.N, 1))

            # Milky Way velocity distribution
            K = vfield0.shape[0]
            w = pm.Dirichlet('w', a=np.ones(K))

            # Set up means and variances:
            meanvs = []
            sigvs = []
            for k in range(K):
                vtmp = pm.Normal(f'vmean{k}', vfield0[k], 10., shape=3)  # HACK

                BoundedNormal = pm.Bound(pm.Normal, lower=1.5, upper=5.3)
                lnstmp = BoundedNormal(f'lns{k}',
                                       np.log(sigma_vfield0[k]), 0.5)
                stmp = pm.Deterministic(f'vsig{k}', tt.exp(lnstmp))

                meanvs.append(vtmp)
                sigvs.append(stmp)

            pvdists = []
            for k in range(K):
                pvtmp = pm.Normal.dist(meanvs[k], sigvs[k], shape=3)
                pvdists.append(pvtmp)
            vxyz = pm.Mixture('vxyz', w=w,
                              comp_dists=pvdists, shape=(self.N, 3))

            # Velocity in tangent plane coordinates
            vtan = tt.batched_dot(self.Ms, vxyz)

            model_pm = vtan[:, :2] / r * pc_mas_yr_per_km_s
            model_rv = vtan[:, 2:3]
            model_y = tt.concatenate((1000 / r, model_pm, model_rv), axis=1)

            pm.Deterministic('model_y', model_y)
            # val = pm.MvNormal('like', mu=model_y, tau=Cinv, observed=y)
            dy = self.ys - model_y
            pm.Potential('chisq',
                         -0.5 * tt.batched_dot(dy,
                                               tt.batched_dot(self.Cinvs, dy)))

        return model

In [ ]:
helper = FieldHelper(subg)
model = helper.get_model(vfield0, sigvfield0)

In [ ]:
with model:
    res = xo.optimize(start={'vxyz': helper.test_vxyz,
                             'r': helper.test_r,
                             'w': np.array([0.5, 0.4, 0.1]),
                             'vmean0': vfield0[0],
                             'vmean1': vfield0[1],
                             'vmean2': vfield0[2],
                             'lns0': np.log(sigvfield0[0]),
                             'lns1': np.log(sigvfield0[1]),
                             'lns2': np.log(sigvfield0[2])})

In [ ]:
res['w']

In [ ]:
res

In [ ]:
from scipy.special import logsumexp
from scipy.stats import multivariate_normal

def logmixprob(vxyz):
    ps = []
    for k in range(3):
        ps.append(multivariate_normal.logpdf(vxyz, res[f'vmean{k}'], 
                                             cov=np.eye(3)*res[f'vsig{k}']**2) + np.log(res['w'][k]))
    return logsumexp(ps, axis=0)

In [ ]:
c = g.get_skycoord()

fig, axes = plt.subplots(1, 3, figsize=(15, 5), sharex=True, sharey=True)
for i in range(3):
    ax = axes[i]

    _x = c.velocity.d_xyz.value[i]
    print(1.5 * np.nanmedian(np.abs(_x - np.nanmedian(_x))),
          np.nanmedian(_x))
    ax.hist(c.velocity.d_xyz.value[i],
            bins=np.linspace(-150, 150, 64), alpha=0.4, 
            label=str(i), color='k', density=True);

    ax.axvline(vthin[i], color='tab:red', linestyle='--')
    ax.axvline(vthik[i], color='tab:green', linestyle='--')
    
    vxyz = np.zeros((1024, 3))
    vxyz[:, i] = np.linspace(-150, 150, vxyz.shape[0])
    ax.plot(vxyz[:, i], np.exp(logmixprob(vxyz)), marker='') # TODO: why is this not normalized right? marginal?

ax.legend(loc='best')

fig.tight_layout()